In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import datetime as dt

#create an empty dataframe for the hospital data broken out by age group and gender
df_hosp = pd.DataFrame(columns=['date', 'age_9_hosp', 'age10_19_hosp', 'age20_29_hosp', 'age30_39_hosp', \
                                'age40_49_hosp', 'age50_59_hosp', 'age60_69_hosp', 'age70_79_hosp', \
                                'age80_plus_hosp', 'ageunk_hosp', 'genderfemale_hosp', 'gendermale_hosp', 'genderunk_hosp'])
df_hosp.set_index(pd.DatetimeIndex(df_hosp['date']), inplace=True)

#location of our data
folder_url = 'http://shulok.com/SD_Covid_data/'

#get the contents of the folder
url = requests.get(folder_url).text
soup = BeautifulSoup(url)
print(soup)


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<html>
<head>
<title>Index of /SD_Covid_data</title>
</head>
<body>
<h1>Index of /SD_Covid_data</h1>
<ul><li><a href="/"> Parent Directory</a></li>
<li><a href="COVID_19_Statistics_San_Diego_County%207-3-2020.csv"> COVID_19_Statistics_San_Diego_County 7-3-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-10-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-10-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-11-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-11-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-13-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-13-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-17-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-17-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-20-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-2

In [2]:
def extract_old_format(df, date):
    df_hosp.loc[date] = [date, df.iloc[0][1], df.iloc[1][1], df.iloc[2][1], df.iloc[3][1], df.iloc[4][1], \
                                   df.iloc[5][1], df.iloc[6][1], df.iloc[7][1], df.iloc[8][1], df.iloc[9][1], \
                                  0, 0, 0]
            
def extract_new_format(df, date):
    df_hosp.loc[date] = [date, df.iloc[1][1], df.iloc[2][1], df.iloc[3][1], df.iloc[4][1], df.iloc[5][1], \
                                   df.iloc[6][1], df.iloc[7][1], df.iloc[8][1], df.iloc[9][1], df.iloc[10][1], \
                                  df.iloc[12][1], df.iloc[13][1], df.iloc[14][1]]

In [4]:
last_date = None
for link in soup.findAll("a"):
    current_link = link.get("href")
    if current_link.endswith('csv') and current_link.count('Hospitalizations') == 1:
        #print('Found CSV: ' + current_link)
        
        #get file date which is right before the file extension
        mylist = current_link.split("%20")
        datestr = mylist[3].split('.')[0]
        #print(datestr)
        
        #convert date string to date object
        date = dt.datetime.strptime(datestr, '%m-%d-%Y')
        if last_date == None or last_date < date:
            last_date = date
         #print(date)
        
        df = pd.read_csv(folder_url + current_link)
        
        if df.columns[0].startswith('Age'):
            #print('Old format')
            extract_old_format(df, date)
            
        elif df.columns[0].startswith('Selected'):
            #print('New format')
            extract_new_format(df, date)

dateformat = "%m-%d-%Y"
filename = 'COVID_19_Hospitalization_Statistics_San_Diego_County %s.csv' % last_date.strftime(dateformat)
print(filename)
df_hosp.to_csv(filename, header=True)            


COVID_19_Hospitalization_Statistics_San_Diego_County 07-02-2020.csv
